# 21. Implement the MT19937 Mersenne Twister RNG

https://cryptopals.com/sets/3/challenges/21

https://en.wikipedia.org/wiki/Mersenne_Twister

In [56]:
class MT19937:

    # Coefficients for MT19937
    (w, n, m, r) = (32, 624, 397, 31)
    a = 0x9908B0DF
    (u, d) = (11, 0xFFFFFFFF)
    (s, b) = (7, 0x9D2C5680)
    (t, c) = (15, 0xEFC60000)
    l = 18
    f = 1812433253

    UMASK = 0xFFFFFFFF & (0xFFFFFFFF << r)       # Limit to 32 bits
    LMASK = 0xFFFFFFFF & (0xFFFFFFFF >> (w - r)) # Limit to 32 bits

    def __init__(self, seed: int = 19650218):
        self.state_array = [0] * self.n  # Array for the state vector
        self.state_index = 0            # Index into state vector array, 0 <= state_index <= n-1
        self.initialize_state(seed)
        self.seed = seed

    def initialize_state(self, seed: int = 19650218):
        self.state_array[0] = seed & 0xFFFFFFFF  # Limit to 32 bits
        for i in range(1, self.n):
            seed = (self.f * (seed ^ (seed >> (self.w - 2))) + i) & 0xFFFFFFFF # Limit to 32 bits
            self.state_array[i] = seed
        self.state_index = 0

    def random(self):
        k = self.state_index               # Current state location
        j = (k - (self.n - 1)) % self.n    # n-1 iterations before

        x = (self.state_array[k] & self.UMASK) | (self.state_array[j] & self.LMASK)
        xA = x >> 1
        if x & 0x00000001:
            xA ^= self.a

        j = (k - (self.n - self.m)) % self.n  # n-m iterations before
        x = self.state_array[j] ^ xA          # Compute next value in the state

        self.state_array[k] = x & 0xFFFFFFFF  # Limit to 32 bits
        k = (k + 1) % self.n                  # Circular indexing
        self.state_index = k

        # Tempering
        y = x ^ (x >> self.u)
        y = y ^ ((y << self.s) & self.b)
        y = y ^ ((y << self.t) & self.c)
        z = y ^ (y >> self.l)
        return z & 0xFFFFFFFF  # Return 32-bit integer

In [57]:
mt_random = MT19937()

for _ in range(10):
    print(mt_random.random())

2325592414
482149846
4177211283
3872387439
1663027210
2005191859
666881213
3289399202
2514534568
3882134983


# 22. Crack an MT19937 seed

https://cryptopals.com/sets/3/challenges/22

In [3]:
import time
import random

def get_MT19937_int32(reallywait=True,tmin=40,tmax=1000):
    '''
    Wait a random number of seconds between 40 and 1000
    Seeds the RNG with the current Unix timestamp
    Waits a random number of seconds again.
    Returns the first 32 bit output of the RNG.
    '''
    sleep1 = random.uniform(tmin,tmax)
    sleep2 = random.uniform(tmin,tmax)
    print(f"Waiting {sleep1:2.2f} seconds...")
    if reallywait:
        time.sleep(sleep1)
        seed = int(time.time())
    else:
        seed = int(time.time()-sleep1-sleep2)
    print(f"Seedind MT19937 with current Unix time ({seed})...")
    mt_random = MT19937(seed)
    print(f"Waiting {sleep2:2.2f} seconds...")
    if reallywait:
        time.sleep(sleep2)
    rnd32 = mt_random.random()
    print(f"Serving random number {rnd32}")
    return rnd32

def crack_MT19937_seed(rnd32):
    print("Attempting to guess MT19937 seed...")
    now = int(time.time())+1
    while True:
        mt_random = MT19937(now)
        _rnd = mt_random.random()
        if rnd32==_rnd:
            print(f"Guessed seed: {now}")
            return now
        # go back in time
        now -= 1

In [4]:
rnd32 = get_MT19937_int32(False)
print()
crack_MT19937_seed(rnd32)

Waiting 130.21 seconds...
Seedind MT19937 with current Unix time (1736599906)...
Waiting 827.66 seconds...
Serving random number 3737348159

Attempting to guess MT19937 seed...
Guessed seed: 1736599906


1736599906

In [5]:
rnd32 = get_MT19937_int32(True,40,100)
print()
crack_MT19937_seed(rnd32)

Waiting 97.84 seconds...
Seedind MT19937 with current Unix time (1736600970)...
Waiting 86.33 seconds...
Serving random number 3350459958

Attempting to guess MT19937 seed...
Guessed seed: 1736600970


1736600970

# 23. Clone an MT19937 RNG from its output

https://cryptopals.com/sets/3/challenges/23

### XOR inverting

Inverting Left-Shift AND XOR (e.g. `y = y ^ ((y << t) & c))`:

This operation involves left-shifting $y$ by $t$ bits, ANDing it with $c$, and XORing it with $y$. To invert it:

* Start with the given $z$.

* Begin with the least significant bits (LSBs), since these are not affected by the left shift.

* Gradually determine higher-order bits, propagating information from lower-order bits.
The loop `for _ in range(32): y = y ^ ((y << t) & c)` ensures the dependencies from lower to higher bits are fully resolved.

In [70]:
def temper(x, u, s, b, t, c, l):
    """
    Applies the tempering process to the input value x.
    
    Parameters:
    - x: The input value to be tempered.
    - u, s, t, l: Shift parameters for the tempering steps.
    - b, c: Masks used in the tempering steps.
    
    Returns:
    - The tempered 32-bit integer value.
    """
    y = x ^ (x >> u)               # Step 1: XOR with right shift
    y = y ^ ((y << s) & b)         # Step 2: XOR with left shift and mask b
    y = y ^ ((y << t) & c)         # Step 3: XOR with left shift and mask c
    z = y ^ (y >> l)               # Step 4: XOR with right shift
    return z & 0xFFFFFFFF          # Ensure 32-bit integer output

def invert_xor_shift(value, shift, mask, direction):
    """
    Inverts an XOR combined with a bitwise shift operation.
    
    Parameters:
    - value: The output value from the tempering step to invert.
    - shift: The number of bits to shift.
    - mask: The mask applied in the original operation (use 0 for no masking).
    - direction: The direction of the shift, either 'left' or 'right'.
    
    Returns:
    - The inverted value.
    """
    result = value
    for _ in range(32):  # Iteratively resolve dependencies for all bits
        if direction == 'right':
            result = value ^ (result >> shift)
        elif direction == 'left':
            result = value ^ ((result << shift) & mask)
    return result

def inverse_temper(z, u, s, b, t, c, l):
    """
    Inverts the tempering function to recover the original value x from z.
    
    Parameters:
    - z: The tempered output value.
    - u, s, t, l: Shift parameters from the tempering function.
    - b, c: Masks used in the tempering function.
    
    Returns:
    - The original input value x.
    """
    # Step 1: Invert the final XOR with a right shift
    y = invert_xor_shift(z, l, 0, 'right')
    
    # Step 2: Invert the XOR with a left shift and mask c
    y = invert_xor_shift(y, t, c, 'left')
    
    # Step 3: Invert the XOR with a left shift and mask b
    y = invert_xor_shift(y, s, b, 'left')
    
    # Step 4: Invert the initial XOR with a right shift
    x = invert_xor_shift(y, u, 0, 'right')
    
    return x & 0xFFFFFFFF  # Ensure 32-bit integer output

In [71]:
# Coefficients for MT19937
(w, n, m, r) = (32, 624, 397, 31)
a = 0x9908B0DF
(u, d) = (11, 0xFFFFFFFF)
(s, b) = (7, 0x9D2C5680)
(t, c) = (15, 0xEFC60000)
l = 18
f = 1812433253

x = 2934351
z = temper(x, u, s, b, t, c, l)
x1 = inverse_temper(z, u, s, b, t, c, l)
print(x,x1)

2934351 2934351


In [72]:
# Initialize target MT19937 generator.
# Use random seed (time, even if time should not be used ;-) )

import time
seed = int(time.time())
mt_random = MT19937(seed)

# let generator run for a while...
import random
for _ in range(int(random.uniform(1,1000))):
    _ = mt_random.random()

In [73]:
# listen for next 624 random number, invert tempering function, store results in state array 
state_array = [ inverse_temper(mt_random.random(), u, s, b, t, c, l) for _ in range(n) ] # n = 624, size of state_array

# create clone generator, use seed=0 to guarantee it'd be different from target one
# align to target uinsg state array
mt_random_clone = MT19937(seed=0)
mt_random_clone.state_array = state_array

# check generators are aligned
print(f"SEED T = {mt_random.seed:10}  SEED C = {mt_random_clone.seed:10}")
print("="*40)
for _ in range(10):
    print(f"Target = {mt_random.random():10}   Clone = {mt_random_clone.random():10}")

SEED T = 1736755485  SEED C =          0
Target =  800446091   Clone =  800446091
Target = 3580834938   Clone = 3580834938
Target = 3873129397   Clone = 3873129397
Target = 3785923601   Clone = 3785923601
Target = 1533888569   Clone = 1533888569
Target = 1359421041   Clone = 1359421041
Target = 2109919503   Clone = 2109919503
Target = 3821632544   Clone = 3821632544
Target = 2013006024   Clone = 2013006024
Target = 1421832573   Clone = 1421832573


### Discussion

> How would you modify MT19937 to make this attack hard? What would happen if you subjected each tempered output to a cryptographic hash?

Some ideas:

**Adding a Cryptographic Hash to Outputs**

By applying a cryptographic hash (like SHA-256 or SHA-3) to each tempered output, one effectively obscures the relationship between the internal state and the output. The cryptographic hash is designed to be one-way, so it becomes computationally infeasible for an attacker to reverse the hash and recover $z$.

**Introduce a Mixing Layer**

After generating the random output $z$, one could pass it through a mixing layer that makes inversion infeasible. For example:

- Use a keyed cryptographic function like AES:
  $\text{output} = \text{AES}_{\text{key}}(z)$
  Without the key, the attacker cannot reverse the output back to \( z \).

- Combine multiple outputs before releasing them:
  $
  \text{output} = (z_1 \oplus z_2) \oplus z_3
  $
  This adds complexity to the recovery process.

**Periodic State Reseeding**

- Periodically reseed the RNG state with entropy from a secure source (e.g., hardware RNG).
- Even if an attacker partially recovers the state, the reseeding process disrupts their ability to predict future outputs.

# 24. Create the MT19937 stream cipher and break it

https://cryptopals.com/sets/3/challenges/24

*8TODO**